In [ ]:
!pip install sqlalchemy==1.3.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


we first load the SQL extension and establish a connection with the database

In [ ]:
%load_ext sql

In [ ]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [ ]:
# !pip install -q pandas==1.1.5

In [ ]:
%sql sqlite:///my_data1.db

'Connected: @my_data1.db'

In [ ]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEX_TBL", con, if_exists='replace', index=False,method="multi")

101

Task 1
Display the names of the unique launch sites in the space mission

In [ ]:
%sql SELECT DISTINCT LAUNCH_SITE FROM SPACEX_TBL;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


Task 2
Display 5 records where launch sites begin with the string 'CCA'

In [ ]:
%sql SELECT * \
    FROM SPACEX_TBL \
    WHERE LAUNCH_SITE LIKE'CCA%' LIMIT 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing _Outcome
04-06-2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
08-12-2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
22-05-2012,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
08-10-2012,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
01-03-2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


Task 3
Display the total payload mass carried by boosters launched by NASA (CRS)

In [ ]:
%sql SELECT SUM(PAYLOAD_MASS__KG_) \
    FROM SPACEX_TBL \
    WHERE CUSTOMER = 'NASA (CRS)';

 * sqlite:///my_data1.db
Done.


SUM(PAYLOAD_MASS__KG_)
45596


Task 4
Display average payload mass carried by booster version F9 v1.1

In [ ]:
%sql SELECT AVG(PAYLOAD_MASS__KG_) \
    FROM SPACEX_TBL \
    WHERE BOOSTER_VERSION = 'F9 v1.1';

 * sqlite:///my_data1.db
Done.


AVG(PAYLOAD_MASS__KG_)
2928.4


Task 5
List the date when the first succesful landing outcome in ground pad was acheived.


In [18]:
%sql SELECT MIN(DATE) FROM SPACEX_TBL WHERE [Landing _Outcome] = 'Success (ground pad)'

 * sqlite:///my_data1.db
Done.


MIN(DATE)
01-05-2017


Task 6
List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [19]:
%sql SELECT PAYLOAD \
FROM SPACEX_TBL \
WHERE [Landing _Outcome] = 'Success (drone ship)' \
AND PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000;

 * sqlite:///my_data1.db
Done.


Payload
JCSAT-14
JCSAT-16
SES-10
SES-11 / EchoStar 105


Task 7
List the total number of successful and failure mission outcomes

In [ ]:
%sql SELECT MISSION_OUTCOME, COUNT(*) as total_number \
FROM SPACEX_TBL \
GROUP BY MISSION_OUTCOME;

 * sqlite:///my_data1.db
Done.


Mission_Outcome,total_number
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


Task 8
List the names of the booster_versions which have carried the maximum payload mass. Use a subquery

In [ ]:
%sql SELECT BOOSTER_VERSION \
FROM SPACEX_TBL \
WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEX_TBL);

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


Task 9
List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.
Note: SQLLite does not support monthnames. So you need to use substr(Date, 4, 2) as month to get the months and substr(Date,7,4)='2015' for year.

In [ ]:
%sql SELECT substr(Date,4,2) as month, DATE,BOOSTER_VERSION, LAUNCH_SITE, [Landing _Outcome] \
FROM SPACEX_TBL \
where [Landing _Outcome] = 'Failure (drone ship)' and substr(Date,7,4)='2015';

 * sqlite:///my_data1.db
Done.


month,Date,Booster_Version,Launch_Site,Landing _Outcome
01,10-01-2015,F9 v1.1 B1012,CCAFS LC-40,Failure (drone ship)
04,14-04-2015,F9 v1.1 B1015,CCAFS LC-40,Failure (drone ship)


Task 10
Rank the count of successful landing_outcomes between the date 04-06-2010 and 20-03-2017 in descending order.

In [ ]:
%sql SELECT [Landing _Outcome], count(*) as count_outcomes \
FROM SPACEX_TBL \
WHERE DATE between '04-06-2010' and '20-03-2017' group by [Landing _Outcome] order by count_outcomes DESC;

 * sqlite:///my_data1.db
Done.


Landing _Outcome,count_outcomes
Success,20
No attempt,10
Success (drone ship),8
Success (ground pad),6
Failure (drone ship),4
Failure,3
Controlled (ocean),3
Failure (parachute),2
No attempt,1
